In [33]:
import numpy as np
import pandas as pd
import json
import os
#from tabulate import tabulate

import matplotlib.pyplot as plt

In [34]:
import git
repo = git.Repo('.', search_parent_directories=True)
repo_path = repo.working_tree_dir

In [35]:
path1 = os.path.join(repo_path,r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\FilesOldBounds\10000")
path2 = os.path.join(repo_path,r"SUSY\SUSY QM\PennyLane\COBYQA\PauliDecomp\VQE\FilesReducedAnsatz\10000")
dataframes = {}
potentials = [f for f in os.listdir(path1) if os.path.isdir(os.path.join(path1, f))]
potentials

['AHO', 'DW', 'QHO']

In [36]:
all_data = []
cutoff_list = [2,4,8,16]

for path in [path1,path2]:

    converged_runs = []
    mean_iters = []
    mean_evals = []
    min_e = []
    delta_min_e = []
    median_e = []
    delta_median_e = []
    mean_e = []
    delta_mean_e = []
    exact_e = []
    time_taken = []

    num_vqe = []
    shots_list = []
    maxiter = []
    cutoffs = []
    ps = []

    for potential in potentials:
        for cutoff in cutoff_list:

            d_path = os.path.join(path, potential, f'{potential}_{cutoff}.json')
        
            try:
                with open(d_path, 'r') as json_file:
                    data = json.load(json_file)
            

                converged_indices = [i for i, success in enumerate(data['success']) if success]

                ps.append(potential)
                cutoffs.append(cutoff)

                num_vqe.append(data['num_VQE'])
                shots_list.append(data['shots'])

                optimzer = data['Optimizer']
                maxiter.append(optimzer['maxiter'])
                
                exact_min_e = np.min(data['exact_eigenvalues'])
                exact_e.append(exact_min_e)

                converged_runs.append(sum(data['success']))

                #iters = [data['num_iters'][i] for i in converged_indices]
                #mean_iters.append(int(np.round(np.mean(iters))))
                mean_iters.append(int(np.round(np.mean(data['num_iters']))))

                #evals = [data['num_evaluations'][i] for i in converged_indices]
                #mean_evals.append(int(np.round(np.mean(evals))))
                #mean_evals.append(int(np.round(np.mean(data['num_evaluations']))))

                #results = [data['results'][i] for i in converged_indices]
                results = data['results']
                min_e.append(np.min(results))
                delta_min_e.append(abs(exact_min_e - np.min(results)))
                median_e.append(np.median(results))
                delta_median_e.append(abs(exact_min_e - np.median(results)))
                mean_e.append(np.mean(results))
                delta_mean_e.append(abs(exact_min_e - np.mean(results)))

                #run_times = [data['run_times'][i] for i in converged_indices]
                #time_series = np.array([pd.Timedelta(t).total_seconds() for t in run_times])
                time_series = np.array([pd.Timedelta(t).total_seconds() for t in data['run_times']])
                mean_vqe_time = np.mean(time_series)
                #mean_vqe_time = pd.Timedelta(seconds=np.mean(time_series))
                time_taken.append(np.mean(mean_vqe_time))
                #time_taken.append(data['parallel_run_time'])

            except Exception as e:
                print(e)
                print("Unable to open file...")

    df = {
    'potential': ps,
    'cutoff': cutoffs,
    'Converged Runs': [f"{c}/100" for c in converged_runs],
    #'Mean Iter': mean_iters,
    #'Mean evals': mean_evals,
    'Mean VQE time (s)': [f"{t:.2f}" for t in time_taken],
    #'E_VQE': [f"{num:.5e}" for num in median_e],
    #'Exact': [f"{num:.5e}" for num in exact_e],
    #'Delta': [f"{num:.5e}" for num in delta_median_e],
    #'VQE min E': [f"{num:.5e}" for num in min_e],
    'Delta min E': [f"{num:.5e}" for num in delta_min_e],
    #'VQE median E': [f"{num:.5e}" for num in median_e],
    'Delta median E': [f"{num:.5e}" for num in delta_median_e],
    #'VQE mean E': [f"{num:.5e}" for num in mean_e],
    'Delta mean E': [f"{num:.5e}" for num in delta_mean_e]
    
    }

    all_data.append(df.copy())

In [37]:
merged = pd.DataFrame(all_data[0]).merge(pd.DataFrame(all_data[1]).reset_index(), on=['potential', 'cutoff'], suffixes=('_path1', '_path2')).drop(columns=['index'])
merged


,potential,cutoff,Converged Runs_path1,Mean VQE time (s)_path1,Delta min E_path1,Delta median E_path1,Delta mean E_path1,Converged Runs_path2,Mean VQE time (s)_path2,Delta min E_path2,Delta median E_path2,Delta mean E_path2
0,AHO,2,100/100,0.20,1.11022e-16,1.11022e-16,1.66533e-16,100/100,0.16,1.11022e-16,1.11022e-16,1.66533e-16
1,AHO,4,100/100,0.37,1.82666e-01,4.90871e-02,4.55319e-01,100/100,0.38,9.78933e-02,5.40104e-02,4.96775e-01
2,AHO,8,100/100,2.58,3.28509e+00,1.66817e-01,4.30246e-02,100/100,3.52,3.01082e+00,8.38033e-01,1.11792e+00
3,AHO,16,100/100,11.87,3.77946e+01,2.69269e+00,1.55106e-02,100/100,12.02,3.78409e+01,1.18003e+00,2.17414e+00
4,DW,2,100/100,0.24,0.00000e+00,0.00000e+00,4.58891e-01,100/100,0.18,0.00000e+00,0.00000e+00,4.05543e-01
5,DW,4,100/100,1.90,6.01650e-02,1.45912e-01,3.55271e-01,100/100,2.48,6.05018e-02,8.30944e-02,2.28904e-01
6,DW,8,100/100,13.58,4.52211e-01,1.17044e-01,6.76229e-01,100/100,8.29,4.97194e-01,5.16507e-01,8.97551e-01
7,DW,16,100/100,49.76,2.19859e+00,6.17636e-02,6.83415e-01,100/100,32.85,2.65345e+00,1.65124e-01,4.90349e-01
8,QHO,2,100/100,0.15,0.00000e+00,0.00000e+00,0.00000e+00,100/100,0.14,0.00000e+00,0.00000e+00,0.00000e+00
9,QHO,4,100/100,0.15,1.11022e-16,1.11022e-16,1.11022e-16,100/100,0.14,1.11022e-16,1.11022e-16,1.11022e-16


In [38]:
exclude = ['potential', 'cutoff', 'Converged Runs_path1','Converged Runs_path2']
cols_to_float = [c for c in merged.columns if c not in exclude]

merged[cols_to_float] = merged[cols_to_float].astype(float)


In [39]:
def compare_metrics(row, pairs, tol=1e-15):
    styles = []
    for pl_col, qk_col in pairs:
        pl_val, qk_val = row[pl_col], row[qk_col]

        if abs(pl_val - qk_val) <= tol:
            styles += ["background-color: #ffffcc", "background-color: #ffffcc"]
        elif pl_val < qk_val:
            styles += ["background-color: green", ""]
        else:
            styles += ["", "background-color: green"]
    return styles


In [40]:
metric_pairs = [("Mean VQE time (s)_path1", "Mean VQE time (s)_path2"), ("Delta min E_path1","Delta min E_path2"), ("Delta median E_path1","Delta median E_path2"), ("Delta mean E_path1","Delta mean E_path2")]
merged.style.apply(lambda r: compare_metrics(r, metric_pairs),
                        subset=[c for pair in metric_pairs for c in pair],
                        axis=1)

,potential,cutoff,Converged Runs_path1,Mean VQE time (s)_path1,Delta min E_path1,Delta median E_path1,Delta mean E_path1,Converged Runs_path2,Mean VQE time (s)_path2,Delta min E_path2,Delta median E_path2,Delta mean E_path2
0,AHO,2,100/100,0.200000,0.000000,0.000000,0.000000,100/100,0.160000,0.000000,0.000000,0.000000
1,AHO,4,100/100,0.370000,0.182666,0.049087,0.455319,100/100,0.380000,0.097893,0.054010,0.496775
2,AHO,8,100/100,2.580000,3.285090,0.166817,0.043025,100/100,3.520000,3.010820,0.838033,1.117920
3,AHO,16,100/100,11.870000,37.794600,2.692690,0.015511,100/100,12.020000,37.840900,1.180030,2.174140
4,DW,2,100/100,0.240000,0.000000,0.000000,0.458891,100/100,0.180000,0.000000,0.000000,0.405543
5,DW,4,100/100,1.900000,0.060165,0.145912,0.355271,100/100,2.480000,0.060502,0.083094,0.228904
6,DW,8,100/100,13.580000,0.452211,0.117044,0.676229,100/100,8.290000,0.497194,0.516507,0.897551
7,DW,16,100/100,49.760000,2.198590,0.061764,0.683415,100/100,32.850000,2.653450,0.165124,0.490349
8,QHO,2,100/100,0.150000,0.000000,0.000000,0.000000,100/100,0.140000,0.000000,0.000000,0.000000
9,QHO,4,100/100,0.150000,0.000000,0.000000,0.000000,100/100,0.140000,0.000000,0.000000,0.000000
